In [2]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Input parameters
capital = 1000

In [64]:
# change the dataframes column names and datatypes
def parse_data(df, exchange_name=""):
    return(
        df.rename(columns={
            "<Ticker>":"ticker",
            '<DTYYYYMMDD>':'date', 
            '<Open>':'open', 
            '<High>':'high', 
            '<Low>':'low', 
            '<Close>':'close',
            '<Volume>':'volume'
        }).assign(
            date = lambda x: pd.to_datetime(x['date'], format="%Y%m%d"),
            exchange = exchange_name
        )
    )

def add_daily_return(df):
    return(
        df.assign(
            daily_return = lambda x: x['close'].pct_change()
        )
    )

In [ ]:
# Read data
hnx = parse_data(pd.read_csv(os.path.join('data', "CafeF.HNX.Upto29.05.2025.csv")), exchange_name="HNX")
hsx = parse_data(pd.read_csv(os.path.join('data', "CafeF.HSX.Upto29.05.2025.csv")), exchange_name= "HOSE")
upcom = parse_data(pd.read_csv(os.path.join('data', "CafeF.UPCOM.Upto29.05.2025.csv")), exchange_name="UPCOM")
index = parse_data(pd.read_csv(os.path.join('data',"CafeF.INDEX.Upto28.05.2025.csv")), exchange_name= "Index")


In [75]:
# Combine all exchanges, calculate daily return
vnstocks = pd.concat([hnx, hsx, upcom], axis=0)[['ticker', 'date', 'close', "exchange"]] \
    .sort_values(['ticker', "date"], ascending=True, ignore_index=True)\
    .groupby('ticker').apply(add_daily_return)\
    .reset_index(drop=True)

In [79]:
vnstocks.set_index("date").resample("M")